<a href="https://colab.research.google.com/github/arifsoul/object-detector-prototype/blob/main/Object_Detector_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A. PENGENALAN**

## 1. Google Colab

## 2. Tensorflow Object Detection API

# **B. PERSIAPAN**

In [ ]:
#@title Jalankan untuk menghubungkan dengan akun *google drive* anda! { vertical-output: true, display-mode: "form" }

from google.colab import drive
drive.mount('/content/drive')



> **Perintah diatas harus dijalankan setiap akan memulai menggunakan *prototype***

> **disarankan menggunakan akun email UNY yang mempunyai kapasitas besar**





## 1. Pembuatan direktori kerja 

Direktori kerja dilakukan dengan menghubungkan akun *google drive*







In [ ]:
#@title Buat direktori kerja! { vertical-output: true, display-mode: "form" }

%cd /content/drive/MyDrive
!git clone https://github.com/arifsoul/pendeteksi-objek.git
%cd /content/drive/MyDrive/pendeteksi-objek
!git clone --quiet https://github.com/tensorflow/models.git

> Tahap pembuatan direktori kerja dilakukan sekali ketika pertama kali menggunakan *prototype* ini atau jika ingin mengganti akun *google drive* anda


> Lewati tahapan ini jika anda sudah membuat direktori kerja sebelumnya



## 2. Pengumpulan *Dataset*

Untuk Mengumpulkan *dataset* diperlukan gambar yang telah dilabeli berdasarkan objek yang ingin di deteksi

> Data label di simpan dalam file berformat ".xml"




In [ ]:
#@title Pelabelan Mandiri { vertical-output: true, display-mode: "form" }
from IPython.display import Image
Image(url='https://github.com/SkalskiP/make-sense/raw/develop/examples/demo-base.gif')


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.makesense.ai/"><img src="https://www.makesense.ai/make-sense-ico-transparent.png"</a>

**Mulai Pelabelan**


  </td>
</table>


<table class="tfo-notebook-buttons" align="right">
  <td>
    <a target="_blank" href="https://github.com/tensorflow/models"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Kode Sumber</a>
  </td>
</table>


#  **C. PELATIHAN *(TRAINING)***
> Proses pelatihan dilakukan untuk memperoleh file *Inference Graph* berformat ".pb" yang akan digunakan untuk *deployment* pada sistem pendeteksi objek

Tahapan yang perlu dilakukan antara lain:

1. *Install Tensorflow Object detection API* dan *packages* yang dibutuhkan
2. Persiapan file *TFrecord*
3. Konfigurasi *training pipeline*
4. *Transfer Learning* dan *Monitoring*
5. Pengujian pendeteksi objek pada gambar menggunakan *checkpoint*
6. *Export Inference Graph* yang telah di training berdasarkan *checkpoint*



## 1. *Install Tensorflow Object detection API* dan *packages* yang dibutuhkan




In [ ]:
#@title Jalankan untuk *Install!* { vertical-output: true, display-mode: "form" }

Versi_Tensorflow = "1" #@param ["1", "2"] {allow-input: true}
version = Versi_Tensorflow +".x"
%tensorflow_version $version
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install tf_slim 
!pip install lvis
!pip install numpy
%cd /content/drive/MyDrive/pendeteksi-objek/models/research/
!protoc object_detection/protos/*.proto --python_out=.
 
import os
os.environ['PYTHONPATH'] += ':/content/drive/MyDrive/pendeteksi-objek/models/research/:/content/drive/MyDrive/pendeteksi-objek/models/research/slim'


## 2. Persiapan file TFrecord

###Masukkan nama folder dataset!

In [ ]:
#@title Ketik nama folder dataset atau pilih jenis pendeteksi dari dataset yang sudah disediakan!  { vertical-output: true, display-mode: "form" }


Pendeteksi = "Masker" #@param ["Masker", "Handphone", "Bola"] {allow-input: true}
repo_url = 'https://github.com/arifsoul/' + Pendeteksi

import os

%cd /content/drive/MyDrive/pendeteksi-objek

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd /content/drive/MyDrive/pendeteksi-objek
!git pull


In [ ]:
#@title *Convert* file .xml ke .csv! { vertical-output: true, display-mode: "form" }
Nama_Folder_Anotasi = "annotations" #@param {type:"string"}


%cd /content/drive/MyDrive/pendeteksi-objek
# Konversikan anotasi pada folder train yang berupa file xml ke dalam satu csv file,
# Buat file `label_map.pbtxt` kepada folder `data/`.
!python xml_to_csv.py -i $Pendeteksi/gambar -o $Pendeteksi/$Nama_Folder_Anotasi/dataset_labels.csv -l $Pendeteksi/$Nama_Folder_Anotasi





In [ ]:
#@title *Split* data *train* dan *test* { vertical-output: true, display-mode: "form" }
%cd /content/drive/MyDrive/pendeteksi-objek/$Pendeteksi/$Nama_Folder_Anotasi

import pandas as pd
import sys
df = pd.read_csv('dataset_labels.csv')
valid_frac = 0.2
valid_size = int(valid_frac * df.filename.nunique())
train_size = df.filename.nunique() - valid_size
valid_df = df.loc[df.filename.isin(df.filename.sample(valid_size))]
train_df = df.loc[df.filename.isin(valid_df.filename.unique())==False]

train_df.to_csv('train_labels.csv', index_label=False, index=False)
valid_df.to_csv('test_labels.csv', index_label=False, index=False)




In [ ]:
#@title Buat file TFrecord Tensorflow { vertical-output: true, display-mode: "form" }
%cd /content/drive/MyDrive/pendeteksi-objek

# Buat file `train.record`
!python generate_tfrecord-$Versi_Tensorflow-.py --csv_input=$Pendeteksi/$Nama_Folder_Anotasi/train_labels.csv --output_path=$Pendeteksi/$Nama_Folder_Anotasi/train.record --img_path=$Pendeteksi/gambar --label_map $Pendeteksi/$Nama_Folder_Anotasi/label_map.pbtxt
# Buat file `test.record`
!python generate_tfrecord-$Versi_Tensorflow-.py --csv_input=$Pendeteksi/$Nama_Folder_Anotasi/test_labels.csv --output_path=$Pendeteksi/$Nama_Folder_Anotasi/test.record --img_path=$Pendeteksi/gambar --label_map $Pendeteksi/$Nama_Folder_Anotasi/label_map.pbtxt


## 3. Konfigurasi *training pipeline*

In [ ]:
#@title Pilih jenis *pre trained model* yang akan digunakan dan buat nama folder untuk *training*! { vertical-output: true, display-mode: "form" }


MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 1
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    },
    'ssd_mobilenet_small_v3': {
        'model_name': 'ssd_mobilenet_v3_small_coco_2019_08_14',
        'pipeline_file': 'ssdlite_mobilenet_v3_small_320x320_coco.config',
        'batch_size': 12
    },
    'ssd_inception_v2_coco': {
        'model_name': 'ssd_inception_v2_coco_2018_01_28',
        'pipeline_file': 'ssd_inception_v2_coco.config',
        'batch_size': 12
    },
    'ssd_mobilenet_large_v3': {
        'model_name': 'ssd_mobilenet_v3_large_coco_2019_08_14',
        'pipeline_file': 'ssdlite_mobilenet_v3_large_320x320_coco.config',
        'batch_size': 512
    }
}

# Pilih model yang akan Anda gunakan 
# Pilih model di dalam `MODELS_CONFIG`.
pre_trained_model = "faster_rcnn_inception_v2" #@param ["ssd_mobilenet_v2", "faster_rcnn_inception_v2", "ssd_inception_v2_coco"] {allow-input: true}


# Nama objek detection model yang digunakan.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# NAma file pipeline pada Tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fit di dalam Colab Tesla K80 GPU memory untuk model yang di pilih.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

%cd /content/drive/MyDrive/pendeteksi-objek

folder_training = "training" #@param {type:"string"}
# Menghapus output konten sebelumnya agar mulai dari fresh kembali (Optional)
!rm -rf {folder_training}
os.makedirs(folder_training, exist_ok=True)

%cd /content/drive/MyDrive/pendeteksi-objek/models/research
import os
import shutil
import glob
import urllib.request
import tarfile

os.makedirs('selected_model', exist_ok=True)
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/drive/MyDrive/pendeteksi-objek/models/research/selected_model/'+selected_model

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint


In [ ]:
#@title Konfigurasi *pipeline* { vertical-output: true, display-mode: "form" }
test_record_fname = '/content/drive/MyDrive/pendeteksi-objek/'+Pendeteksi+'/'+Nama_Folder_Anotasi+'/test.record'
train_record_fname = '/content/drive/MyDrive/pendeteksi-objek/'+Pendeteksi+'/'+Nama_Folder_Anotasi+'/train.record'
label_map_pbtxt_fname = '/content/drive/MyDrive/pendeteksi-objek/'+Pendeteksi+'/'+Nama_Folder_Anotasi+'/label_map.pbtxt'

import os
pipeline_fname = os.path.join('/content/drive/MyDrive/pendeteksi-objek/models/research/object_detection/samples/configs', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())


import re

num_classes = get_num_classes('/content/drive/MyDrive/pendeteksi-objek/Masker/annotations/label_map.pbtxt')
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord file train dan test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set jumlah classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    # Set jumlah contoh (jumlah gambar).
    s = re.sub('num_examples: [0-9]+',
               'num_examples: {}'.format(num_examples), s)
    

    f.write(s)

## 4. *Transfer Learning* dan *Monitoring*





In [ ]:
#@title Inisialisasi Tensorboard { vertical-output: true, display-mode: "form" }
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

LOG_DIR = folder_training
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')



In [ ]:
#@title *Monitoring* menggunakan *link* dibawah ini { vertical-output: true, display-mode: "form" }
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
%cd /content/drive/MyDrive/pendeteksi-objek
#@title Mulai *Training* { vertical-output: true, display-mode: "form" }
# Jumlah training step.
num_steps =  1000000#@param {type:"integer"}

# Jumlah evaluation step.
num_eval_steps =  50#@param {type:"integer"}

# Jumlah sample di dalam folder "test".
num_examples =  45#@param {type:"integer"}

!python /content/drive/MyDrive/pendeteksi-objek/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={folder_training} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

## 5. Pengujian pendeteksi objek pada gambar menggunakan *checkpoint*

## 6. Pembuatan *Inference Graph* dari data *training*

In [ ]:
#@title Buat folder untuk menyimpan *inference graph!* { vertical-output: true, display-mode: "form" }
Nama_Folder_Inference_Graph = "graph1" #@param {type:"string"}

import re
import numpy as np


lst = os.listdir(folder_training)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(folder_training, last_model)
print(last_model_path)
!python /content/drive/MyDrive/pendeteksi-objek/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory=/content/drive/MyDrive/pendeteksi-objek/inference_graph/$Nama_Folder_Inference_Graph \
    --trained_checkpoint_prefix={last_model_path}

# **D. PENYEBARAN *(DEPLOYMENT)***

> File *inference graph* yang telah diperoleh pada tahap *training* akan dibaca pada program sistem pendeteksi objek untuk mengenali pola gambar yang telah ditentukan

>Berikut merupakan *prototype* sistem pendeteksi objek yang dapat bekerja secara *realtime* menggunakan webcam


In [ ]:
#@title Object Detector { vertical-output: true, display-mode: "form" }
Nama_Folder_Inference_Graph = "graph1" #@param {type:"string"}
Pendeteksi = "Masker" #@param ["Masker", "Bola", "Handphone"] {allow-input: true}
Nama_Folder_Anotasi = "annotations" #@param {type:"string"}

# NOTE: Pieces taken from https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb
# The github.com/tensorflow/models is distributed under the Apache 2.0 license.

import numpy
import tensorflow as tf
import base64 
import html
import io
import time

# setup path so that model can be found.
import sys
sys.path.append('/content/drive/MyDrive/pendeteksi-objek/models/research/object_detection')
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils

# Taken from https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=SucxddsPhOmj
from IPython.display import display, Javascript
from google.colab.output import eval_js
import numpy
import PIL.Image

checkpoint_name = '/content/drive/MyDrive/pendeteksi-objek/'+Nama_Folder_Inference_Graph
#checkpoint = '{0}.ckpt'.format(checkpoint_name)

with tf.get_default_graph().as_default() as graph:  

  # This section builds a "graph" in TensorFlow to explain how to process the data.
  jpeg_input_tensor = tf.placeholder(tf.string, ())  # We will provide a JPEG to TF.

  # First, instruct TF to decode the JPEG string into a matrix.
  image = tf.image.decode_image(jpeg_input_tensor)
  image_tensor = tf.expand_dims(image, 0)

  # Load the  graph from disk.
  ssd_graph = tf.GraphDef()
  with open('{}/frozen_inference_graph.pb'.format(checkpoint_name), 'rb') as f:
    ssd_graph.ParseFromString(f.read())
    
  # Tell TensorFlow we would like to inspect these parts of the network.
  output_names = ['num_detections:0', 
                  'detection_boxes:0', 
                  'detection_scores:0',
                  'detection_classes:0']
  ops = dict(zip(output_names, tf.graph_util.import_graph_def(
      ssd_graph, 
      input_map={'image_tensor': image_tensor},
      return_elements=output_names)))
    
  # Also extract the decoded image from the network to draw bounding boxes.
  ops['image'] = image



def run_detection(sess, img):
  """Run one detection round."""
  output_dict = sess.run(ops, feed_dict={jpeg_input_tensor: img})

  # all outputs are float32 numpy arrays, so convert types as appropriate
  output_dict['num_detections'] = int(output_dict['num_detections:0'][0])
  output_dict['detection_classes'] = output_dict['detection_classes:0'][0].astype(numpy.int64)
  output_dict['detection_boxes'] = output_dict['detection_boxes:0'][0]
  output_dict['detection_scores'] = output_dict['detection_scores:0'][0]
  
  return output_dict




def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def take_photo(label, img_data):
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

category_index = label_map_util.create_category_index_from_labelmap('/content/drive/MyDrive/pendeteksi-objek/'+Pendeteksi+'/'+Nama_Folder_Anotasi+'/label_map.pbtxt',
    use_display_name=True)
    
with tf.Session() as sess:
  start_input()

  label_html = 'Detctioning'
  img_data = ''
  while True:
    capture_start = time.time()
    js_reply = take_photo(label_html, img_data)
    capture_end = time.time()
    if not js_reply:
      break

    # Javascript returns a data URL, like:
    #     data: image/jpeg;base64,<base-64 encoded data>
    # To use the image, decode the base-64 encoded part and treat it as a JPEG.
    jpeg_input = base64.b64decode(js_reply['img'].split(',')[1])
    result = run_detection(sess, jpeg_input)
    detect_end = time.time()
    
    # To reduce transfer sizes, we send just the bounding boxes drawn on a 
    # transparent PNG. Here, we create a blank PNG.
    rgb_shape = result['image'].shape
    rgba_shape = list(rgb_shape)[0:2] + [4]
    image_np = numpy.zeros(rgba_shape, dtype=numpy.uint8)
    
    # Draw the bounding boxes in the RGB channels.
    visualization_utils.visualize_boxes_and_labels_on_image_array(
      image_np[:, :, 0:3],  # sub-select RGB channels only; alpha is done below.
      result['detection_boxes'],
      result['detection_classes'],
      result['detection_scores'],
      category_index,
      instance_masks=result.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=3)
    
    # To be visible, the alpha channel also needs to be edited. Set the alpha
    # channel to 255 (fully opaque) wherever anything was drawn.
    image_t = image_np.transpose()
    max_color = numpy.maximum(numpy.maximum(image_t[0], image_t[1]), image_t[2])
    image_t[3] = numpy.clip(max_color, 0, 1) * 255
    viz_end = time.time()

    # Save the image as a PNG in memory and assemble a data URL.
    im = PIL.Image.fromarray(image_np, 'RGBA')
    iobuf = io.BytesIO()
    im.save(iobuf, format='png')
    img_data = 'data:image/png;base64,{}'.format(
      (str(base64.b64encode(iobuf.getvalue()), 'utf-8')))

    perf_measures = {
        'server': (
            ('take_photo', capture_end - capture_start),
            ('run_detection', detect_end - capture_end),
            ('visualize', viz_end - detect_end)
        ),
        'js': (
            ('create', js_reply['create']),
            ('show', js_reply['show']),
            ('capture', js_reply['capture']),
        ),
    }
    
    label_text = 'img size: {}b\ntime:\n  server: {}\n  js: {}'.format(
        len(js_reply['img']),
        ', '.join('{}: {:2.3f}s'.format(*x) for x in perf_measures['server']),
        ', '.join('{}: {:2.3f}s'.format(x[0], x[1] / 1000) for x in perf_measures['js']),
    )
    
    label_html = html.escape(label_text).replace('\n', '<br/>')

print('Finished')



#**Sumber Referensi**

```
Mart\’\in~Abadi, Ashish~Agarwal, Paul~Barham, Eugene~Brevdo, Zhifeng~Chen, Craig~Citro, … Xiaoqiang~Zheng. (2015). {TensorFlow}: Large-Scale Machine Learning on Heterogeneous Systems.

Skalski, P. (2019). Make Sense.



```



